In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
    
  # Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
    
    # Configure OS routines
import os
    
    # Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
    
    #### FIXED #####\
    # change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter
    
    ###########################
    # Data Manipulation / Model
    ###########################
    # FIXED update with your username and password and CRUD Python module name
    
username = "aacuser"
password = "password"
    
    # Connect to database via CRUD Module
db = AnimalShelter(username, password)
    
    # class read method must support return of list object and accept projection json input
    # sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readAll({}))
    
    # MongoDB v5+ is going to return the '_id' column and that is going to have an
    # invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
    # it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
    # inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
    
    ## Debug
    # print(len(df.to_dict(orient='records')))
    # print(df.columns)
    
    
    #########################
    # Dashboard Layout / View
    #########################
app = JupyterDash('Simple Example')
    
    #FIXED Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#App layout that includes image and identifier

app.layout = html.Div([
   # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Laura McAroy CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
           #options for radio item filters
        dcc.RadioItems(
            id='filter-type',
            options=[{'label':'Water Rescue', 'value':'WTR'},
                {'label':'Mountain/Wilderness Rescue', 'value':'MWR'},
                {'label':'Disaster Rescue', 'value':'DR'},
                {'label':'Reset - remove filters' , 'value':'RESET'}
                    
            ],
            value='RESET',
            labelStyle={'display':'inline-block'}
        )
    ),
   
    
    
    html.Hr(),
    dt.DataTable(id='datatable-id',
                                columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                             data=df.to_dict('records'),
    #FIXED: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 
    column_selectable = False,
    row_selectable = "single",
    sort_action = "native",
    filter_action = "native",
    editable = False,
    sort_mode = "multi",
    row_deletable = False,
    selected_rows = [],
    selected_columns = [],
    page_action="native",
    page_current=0,
    page_size = 10,
                             
),
                            
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
             style={'display' : 'flex'},
                 children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
    
                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
            ])
])
    
    
    #############################################
    # Interaction Between Components / Controller
    #############################################
    
        
    
@app.callback(Output('datatable-id','data'),
              Output('datatable-id', 'columns'),
                  [Input('filter-type', 'value')])
    
def update_dashboard(filter_type):
    ## FIXED Add code to filter interactive data table with MongoDB queries
 # Filter results for ideal Water Rescue Animals
        if filter_type == 'WTR':
          df = pd.DataFrame(list(db.read({'$and':[{'sex_upon_outcome':'Intact Female'},
                                           {'$or':[
                                               {'breed': 'Labrador Retriever Mix'},
                                               {'breed': 'Chesa Bay Retr Mix'},
                                               {'breed': 'Newfoundland Mix'}]
                                           },
                                           {'$and':[
                                               {'age_upon_outcome_in_weeks': {'$gte' : 26}},
                                               {'age_upon_outcome_in_weeks': {'$lte' : 156}}]
                                           }
                                                    ]}
                                             )))
        
 # Filter results for ideal Mountain / Wilderness Animals 
        
        elif filter_type == 'MWR':
            df=pd.DataFrame(list(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'German Shepard'},
                                                         {'breed': 'Alaskan Malamute'},
                                                         {'breed': 'Old English Sheepdog'},
                                                         {'breed': 'Rottweiler'},
                                                         {'breed': 'Siberian Husky'}]
                                                     },
                                                     {'$and':[
                                                         {'age_upon_outcome_in_weeks': {'$gte' : 26}},
                                                         {'age_upon_outcome_in_weeks': {'$lte' : 156}}]
                                                     }
                                                       ]}
                                           )))
 
 # Filter results for ideal Disaster Rescue Animals
        
        elif filter_type == 'DR':
            df=pd.DataFrame(list(db.read({'$and':[{'sex_upon_outcome': 'Intact Male'},
                                                   {'$or': [
                                                       {'breed': 'Doberman Pinscher'},
                                                       {'breed': 'German Shepard'},
                                                       {'breed': 'Golden Retriever'},
                                                       {'breed': 'Bloodhound'},
                                                       {'breed': 'Rottweiler'}]
                                                   },
                                                   {'$and' :[
                                                       {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                       {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                   }
                                                      ]}
                                           )))
# Reset and clear all filters
        elif filter_type == 'RESET':
            df = pd.DataFrame.from_records(db.read({}))
           
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data, columns)
        #This callback will highlight a cell on the data table when the user selects it
@app.callback(
        Output('datatable-id', 'style_data_conditional'),
        [Input('datatable-id', 'selected_columns')]
    )

def update_styles(selected_columns):
    return [{
        'if':{ 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
           

    
    # Display the breeds of animal based on quantity represented in
    # the data table
@app.callback(
        Output('graph-id', "children"),
        [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    #imports data
    dff= pd.DataFrame.from_dict(viewData)
    #creates values for graph
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()

#Creates pie chart   
    return [
            dcc.Graph(            
                figure = px.pie(data_frame=dff, values = values, names = names,
                                
                                width = 700, height = 400)
                               
                )    
        ]
        

    
    
    # This callback will update the geo-location chart for the selected data entry
    # derived_virtual_data will be the set of data available from the datatable in the form of 
    # a dictionary.
    # derived_virtual_selected_rows will be the selected row(s) in the table in the form of
    # a list. For this application, we are only permitting single row selection so there is only
    # one value in the list.
    # The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])

def update_map(viewData, selected_rows, selected_columns): 
   
 #imports data
    dff = pd.DataFrame.from_dict(viewData)
   
    if selected_rows==[]:
        selected_rows = [0]
        

   
            
        # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]], children=[
                dl.Tooltip(dff.iloc[selected_rows[0],4]),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(dff.iloc[selected_rows[0],9]),
                    html.H4("Breed"),
                    html.P(dff.iloc[selected_rows[0],4])
                ])
            ])
        ])
    ]
               
    
    
    
app.run_server(debug=True)
   
  
 


Dash app running on http://127.0.0.1:8549/
